<a href="https://colab.research.google.com/github/Huxwell/ColabNNs/blob/main/mobilenet_eggs_WORKING_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from zipfile import ZipFile
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
zip_path = '/content/drive/MyDrive/OVO-TECH/eggs_pascal_wa0000-19_no_08_classify_from_bbs_augmented.zip'
extract_path = '/content/dataset_train'

# Extract the ZIP file
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


zip_path = '/content/drive/MyDrive/OVO-TECH/eggs_pascal_wa0008_UPDATED_classify_from_bbs.zip'
extract_path = '/content/dataset_test'

# Extract the ZIP file
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

# Transformations for the train
transform_train = transforms.Compose([transforms.Resize(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Transformations for the test
transform_test = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Load the dataset from the folders
dataset_train = torchvision.datasets.ImageFolder('/content/dataset_train/eggs_pascal_wa0000-19_no_08_classify_from_bbs_augmented', transform=transform_train)
dataset_test = torchvision.datasets.ImageFolder('/content/dataset_test/eggs_pascal_wa0008_UPDATED_classify_from_bbs', transform=transform_test)

# Create dataloaders - they will provide us with batches, which is necessary when training the network
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=True)


In [ ]:
import torch.optim as optim
import torchvision.models as models

# Load the pretrained model
model = models.mobilenet_v2(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Change the last layer to match the number of classes in your dataset
# You have 3 classes: a_yolk, background, b_yolk
num_classes = 3
model.classifier[1] = torch.nn.Linear(model.last_channel, num_classes)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
import torch
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU if available
model = model.to(device)

# Training function
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Training
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        # Create a progress bar for added visibility
        progress_bar = tqdm(enumerate(dataloader_train), total=len(dataloader_train))
        for i, (inputs, labels) in progress_bar:
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            if i % 10 == 0 and i > 0:  # print every 10 mini-batches
                batch_loss = running_loss / ((i + 1) * inputs.size(0))
                batch_acc = running_corrects.double() / ((i + 1) * inputs.size(0))
                progress_bar.set_description(f'Loss: {batch_loss:.4f}, Acc: {batch_acc:.4f}')

        epoch_loss = running_loss / len(dataset_train)
        epoch_acc = running_corrects.double() / len(dataset_train)
        print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))


        # Validation
        model.eval()  # Set model to evaluate mode
        running_loss = 0.0
        running_corrects = 0
        with torch.no_grad():
            for inputs, labels in dataloader_test:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                running_loss += criterion(outputs, labels).item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataset_test)
        epoch_acc = running_corrects.double() / len(dataset_test)
        print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    print('Finished Training')

# Call the training function
train_model(model, criterion, optimizer, num_epochs=2)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluate mode
    true_labels = []
    pred_labels = []
    images = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Save the images
            images.extend(inputs.cpu().numpy())

            true_labels += labels.tolist()
            pred_labels += preds.tolist()

    print('Classification Report:')
    print(classification_report(true_labels, pred_labels, labels=[0, 1, 2], target_names=['a_yolk', 'background', 'b_yolk']))

    return images, true_labels, pred_labels

# Call the evaluation function
images, true_labels, pred_labels = evaluate_model(model, dataloader_test)



In [ ]:
assert 1==0

In [ ]:
import torchvision.transforms.functional as F
import numpy as np

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

from collections import defaultdict

def show_images(images, true_labels, pred_labels, class_names):
    correct = np.where(np.array(true_labels) == np.array(pred_labels))[0]
    incorrect = np.where(np.array(true_labels) != np.array(pred_labels))[0]

    correct_counts = defaultdict(int)
    incorrect_counts = defaultdict(int)

    correct_imgs = []
    incorrect_imgs = []

    for idx in correct:
        if correct_counts[true_labels[idx]] < 4:
            correct_imgs.append((images[idx], true_labels[idx], pred_labels[idx]))
            correct_counts[true_labels[idx]] += 1

    for idx in incorrect:
        if incorrect_counts[true_labels[idx]] < 4:
            incorrect_imgs.append((images[idx], true_labels[idx], pred_labels[idx]))
            incorrect_counts[true_labels[idx]] += 1

    print(f"Found {len(correct_imgs)} correctly classified images")
    print(f"Found {len(incorrect_imgs)} incorrectly classified images")

    # Show the correctly classified images
    print("Showing Correctly Classified Images")
    fig = plt.figure(figsize=(20, 10))
    for i in range(len(correct_imgs)):
        ax = plt.subplot(1, len(correct_imgs), i + 1)
        imshow(correct_imgs[i][0], title=f"Predicted: {class_names[correct_imgs[i][2]]}, Actual: {class_names[correct_imgs[i][1]]}")

    # Show the incorrectly classified images
    print("Showing Incorrectly Classified Images")
    fig = plt.figure(figsize=(20, 10))
    for i in range(len(incorrect_imgs)):
        ax = plt.subplot(1, len(incorrect_imgs), i + 1)
        imshow(incorrect_imgs[i][0], title=f"Predicted: {class_names[incorrect_imgs[i][2]]}, Actual: {class_names[incorrect_imgs[i][1]]}")
class_names = ['a_yolk', 'background', 'b_yolk']
show_images(images, true_labels, pred_labels, class_names)

